# 如何解析 YAML 输出

:::info 先决条件

本指南假设您熟悉以下概念：
- [Chat 模型](/docs/concepts/chat_models)
- [输出解析器](/docs/concepts/output_parsers)
- [提示模板](/docs/concepts/prompt_templates)
- [结构化输出](/docs/how_to/structured_output)
- [将可运行对象链接在一起](/docs/how_to/sequence/)

:::

来自不同提供商的大语言模型 (LLM) 通常有不同的优势，这取决于它们所训练的具体数据。这也意味着有些模型可能在生成除 JSON 以外的其他格式的输出时“更好”且更可靠。

此输出解析器允许用户指定任意模式，并查询 LLM 以获取符合该模式的输出，使用 YAML 来格式化其响应。

:::note
请记住，大型语言模型是易变的抽象！您需要使用具有足够容量的 LLM 来生成格式良好的 YAML。
:::

In [ ]:
%pip install -qU langchain langchain-openai

import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

我们使用 [Pydantic](https://docs.pydantic.dev) 和 [`YamlOutputParser`](https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.yaml.YamlOutputParser.html#langchain.output_parsers.yaml.YamlOutputParser) 来声明我们的数据模型，并为模型提供更多关于它应该生成何种类型 YAML 的上下文：

In [4]:
from langchain.output_parsers import YamlOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


model = ChatOpenAI(temperature=0)

# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = YamlOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

Joke(setup="Why couldn't the bicycle find its way home?", punchline='Because it lost its bearings!')

解析器将自动解析输出的 YAML 并创建一个包含数据的 Pydantic 模型。我们可以看到解析器的 `format_instructions`，它会被添加到提示中：

In [5]:
parser.get_format_instructions()

'The output should be formatted as a YAML instance that conforms to the given JSON schema below.\n\n# Examples\n## Schema\n```\n{"title": "Players", "description": "A list of players", "type": "array", "items": {"$ref": "#/definitions/Player"}, "definitions": {"Player": {"title": "Player", "type": "object", "properties": {"name": {"title": "Name", "description": "Player name", "type": "string"}, "avg": {"title": "Avg", "description": "Batting average", "type": "number"}}, "required": ["name", "avg"]}}}\n```\n## Well formatted instance\n```\n- name: John Doe\n  avg: 0.3\n- name: Jane Maxfield\n  avg: 1.4\n```\n\n## Schema\n```\n{"properties": {"habit": { "description": "A common daily habit", "type": "string" }, "sustainable_alternative": { "description": "An environmentally friendly alternative to the habit", "type": "string"}}, "required": ["habit", "sustainable_alternative"]}\n```\n## Well formatted instance\n```\nhabit: Using disposable water bottles for daily hydration.\nsustainabl

你可以并且应该尝试在提示的其他部分添加你自己的格式化提示，以增强或替换默认指令。

## 后续步骤

你已经学会了如何提示模型返回 YAML。接下来，请查看关于获取结构化输出的[更广泛指南](/docs/how_to/structured_output)，了解其他相关技术。